In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## Step 1: Data Loading

import pandas as pd

train = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
test = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')

print("Train shape:", train.shape)
print("Test shape:", test.shape)
train.head()


In [ ]:
## Step 2: Exploratory Data Analysis (EDA)

print(train.columns)
print(train.isnull().sum())


In [ ]:
## Step 3: Feature Engineering

print(train[['winner_model_a','winner_model_b','winner_tie']].sum())


In [ ]:
## Step 4: Model Training

train['prompt_len'] = train['prompt'].apply(len)
train['response_a_len'] = train['response_a'].apply(len)
train['response_b_len'] = train['response_b'].apply(len)

print(train[['prompt_len','response_a_len','response_b_len']].describe())


In [ ]:
## Step 5: Validation

import matplotlib.pyplot as plt
train[['prompt_len','response_a_len','response_b_len']].hist(bins=50, figsize=(10,4))
plt.show()


In [ ]:
## Step 6: Prediction and Submission

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

y = (train['winner_model_a'] == 1).astype(int)

X = train[['prompt_len', 'response_a_len', 'response_b_len']]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

print("Validation Accuracy:", model.score(X_valid, y_valid))


In [ ]:
## Step 7: Observations

test_X = test[['prompt', 'response_a', 'response_b']].copy()

test_X['prompt_len'] = test_X['prompt'].apply(len)
test_X['response_a_len'] = test_X['response_a'].apply(len)
test_X['response_b_len'] = test_X['response_b'].apply(len)

test_pred = model.predict_proba(test_X[['prompt_len', 'response_a_len', 'response_b_len']])[:, 1]

submission = pd.DataFrame({
    'id': test['id'],
    'winner_model_a': test_pred
})

submission.to_csv('submission.csv', index=False)
print("Saved submission.csv.")
